## migrate TELGEN data to TEAL

In [1]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial
import random
import pickle 

### Resource Allocation

#### input: for one graph   
#### G(V, E, c): random graph (strongly connected)
#### (s, t, d) \in [S, T, D]  
#### for every (s, t, d), there is a set p \in Pd (k-shortest path algorithm (4/5/6))

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

## teal pickle for traffic matrix

In [3]:
# the data should be put into the TEAL codebase to run
root = 'home/user/TEAL/'

In [4]:
##### gen teal pickle file for traffic matrix #####
# Specify the path to your pkl.gz file ins 0
# teal_harp_migrate: save the raw generated for teal and harp to migrate

from collections import defaultdict

ins_num = [0]

num_nodes = [1739] 

for ins in range(len(ins_num)):
    file_path = 'raw/raw/teal_harp_migrate/instance_' + str(ins_num[ins]) + '_stds.pkl'

    # Open the file in binary mode and use gzip to decompress it
    with open(file_path, 'rb') as f:
        # Load the pickled data
        data = pickle.load(f)
    print('orin data len:', len(data))

    matrices = np.zeros((len(data), num_nodes[ins], num_nodes[ins]))
    for i in range(len(data)): 
        for j in range(len(data[i])):
            matrices[i][int(data[i][j][0][0])][int(data[i][j][0][1])] = data[i][j][1]

    for n in range(len(data)):
        with open(root+'traffic-matrices/ASN1739/ASN'+str(num_nodes[ins])+'.json_ASN_'+ str(n) + '_1.0_traffic-matrix.pkl','wb') as f:
            pickle.dump(matrices[n], f)

In [5]:
import json

f = open('raw/asn_graph/ASN2k.json')
G = json.load(f)

asn_graph = nx.DiGraph()
asn_graph.add_nodes_from([i['id'] for i in G['nodes']])
asn_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])

print('Strongly connected:', nx.is_strongly_connected(asn_graph))
print('# of nodes and edges:', asn_graph.number_of_nodes(), asn_graph.number_of_edges())
print('Weighted:', nx.is_weighted(asn_graph))

asn_graph_weight = nx.DiGraph()
asn_graph_weight.add_nodes_from([i['id'] for i in G['nodes']])
for i in G['links']:
    asn_graph_weight.add_edge(i['source'], i['target'], weight=i['capacity'])
print('Strongly connected:', nx.is_strongly_connected(asn_graph_weight))
print('# of nodes and edges:', asn_graph_weight.number_of_nodes(), asn_graph_weight.number_of_edges())
print('Weighted:', nx.is_weighted(asn_graph_weight))

# nx.draw(asn_graph, with_labels=True, node_color='lightgreen', arrows=True)

## teal pickle for path dic

In [6]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

import json
num_nodes = [1739] 

k = 4
Pd = {}
for i in asn_graph.nodes():
    for j in asn_graph.nodes():
        if i == j:
            continue
        else:
            k_paths = k_shortest_paths(asn_graph, i, j, k=k)
            Pd[(i, j)] = k_paths
with open(root+'traffic-matrices/ASN1739/ASN'+str(num_nodes[0])+'.json-'+str(k)+'-paths_edge-disjoint-True_dist-metric-min-hop-dict.pkl','wb') as f:
    pickle.dump(Pd, f)